# Environmetn Setup

In [ ]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

In [ ]:

!pip install -qU "langchain[openai]" # select chat model OpenAI

In [3]:
!pip install -qU langchain-openai # select embeddings model OpenAI

In [4]:

!pip install -qU langchain-community # select vector store FAISS

In [ ]:

!pip install jq

In [ ]:

!pip install faiss-cpu

In [ ]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [8]:
from google.colab import drive
drive.mount('/content/drive')
%cd  /content/drive/MyDrive/ECE1508_Project/Codes

Mounted at /content/drive
/content/drive/MyDrive/ECE1508_Project/Codes


# Load Test Doc

In [11]:
file_path="test_single_doc.json"

In [12]:
import json
from langchain_community.document_loaders import JSONLoader

def metadata_fuc(record:dict,metadata:dict)->dict:
  questions=record.get("questions",[])
  metadata["question_texts"]=[q.get("question_text") for q in questions]
  metadata["document_url"]=record.get("document_url")
  return metadata

loader=JSONLoader(
  file_path=file_path,
  jq_schema=".",
  content_key="document_text",
  metadata_func=metadata_fuc
)

documents=loader.load()

In [20]:
print(documents[0].page_content)

Maze Runner : the Death Cure - wikipedia <H1> Maze Runner : the Death Cure </H1> Jump to : navigation , search For the book the film is based on , see The Death Cure . <Table> <Tr> <Th_colspan="2"> Maze Runner : The Death Cure </Th> </Tr> <Tr> <Td_colspan="2"> Theatrical release poster </Td> </Tr> <Tr> <Th> Directed by </Th> <Td> Wes Ball </Td> </Tr> <Tr> <Th> Produced by </Th> <Td> <Ul> <Li> Ellen Goldsmith - Vein </Li> <Li> Wyck Godfrey </Li> <Li> Marty Bowen </Li> <Li> Joe Hartwick , Jr . </Li> <Li> Wes Ball </Li> <Li> Lee Stollman </Li> </Ul> </Td> </Tr> <Tr> <Th> Screenplay by </Th> <Td> T.S. Nowlin </Td> </Tr> <Tr> <Th> Based on </Th> <Td> The Death Cure by James Dashner </Td> </Tr> <Tr> <Th> Starring </Th> <Td> <Ul> <Li> Dylan O'Brien </Li> <Li> Kaya Scodelario </Li> <Li> Thomas Brodie - Sangster </Li> <Li> Nathalie Emmanuel </Li> <Li> Giancarlo Esposito </Li> <Li> Aidan Gillen </Li> <Li> Walton Goggins </Li> <Li> Ki Hong Lee </Li> <Li> Barry Pepper </Li> <Li> Will Poulter </Li>

In [14]:
print(documents[0].metadata.get("question_texts"))

['when did the new maze runner movie come out', 'when is the death cure coming out in ireland', 'when is the death cure released in the uk']


# Proposition Sample

Reference:https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb


 The proposition promt come from: https://smith.langchain.com/hub/wfh/proposal-indexing?organizationId=50995362-9ea0-4378-ad97-b4edae2f9f22


In [50]:
from langchain.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain.chains import create_extraction_chain
from typing import Optional, List
from langchain.chains import create_extraction_chain_pydantic
from pydantic import BaseModel
from langchain import hub
import re
from langchain_core.pydantic_v1 import BaseModel
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate


In [83]:
#Get test HTML text
test_html = documents[0].page_content
# Find all <h2> tag positions (case-insensitive)
matches = list(re.finditer(r'(?i)<h2[^>]*>', test_html))

test_sections = []

# 1. Add content before the first <h2>
if matches and matches[0].start() > 0:
    test_sections.append(test_html[:matches[0].start()].strip())

# 2. Add sections between <h2> tags
for i in range(len(matches)):
    start = matches[i].start()
    end = matches[i+1].start() if i + 1 < len(matches) else len(test_html)
    section = test_html[start:end].strip()
    test_sections.append(section)

# 3. Add content after the last </h2> (if needed)
last_h2_end = matches[-1].end() if matches else 0
post_h2_text = test_html[last_h2_end:].strip()
if post_h2_text:
    test_sections.append(post_h2_text)

# Print results
for i, section in enumerate(test_sections):
    print(f"--- Section {i} ---")
    print(section)

--- Section 0 ---
Maze Runner : the Death Cure - wikipedia <H1> Maze Runner : the Death Cure </H1> Jump to : navigation , search For the book the film is based on , see The Death Cure . <Table> <Tr> <Th_colspan="2"> Maze Runner : The Death Cure </Th> </Tr> <Tr> <Td_colspan="2"> Theatrical release poster </Td> </Tr> <Tr> <Th> Directed by </Th> <Td> Wes Ball </Td> </Tr> <Tr> <Th> Produced by </Th> <Td> <Ul> <Li> Ellen Goldsmith - Vein </Li> <Li> Wyck Godfrey </Li> <Li> Marty Bowen </Li> <Li> Joe Hartwick , Jr . </Li> <Li> Wes Ball </Li> <Li> Lee Stollman </Li> </Ul> </Td> </Tr> <Tr> <Th> Screenplay by </Th> <Td> T.S. Nowlin </Td> </Tr> <Tr> <Th> Based on </Th> <Td> The Death Cure by James Dashner </Td> </Tr> <Tr> <Th> Starring </Th> <Td> <Ul> <Li> Dylan O'Brien </Li> <Li> Kaya Scodelario </Li> <Li> Thomas Brodie - Sangster </Li> <Li> Nathalie Emmanuel </Li> <Li> Giancarlo Esposito </Li> <Li> Aidan Gillen </Li> <Li> Walton Goggins </Li> <Li> Ki Hong Lee </Li> <Li> Barry Pepper </Li> <Li> 

In [88]:
obj = hub.pull("wfh/proposal-indexing")
llm = ChatOpenAI(model='gpt-4-1106-preview', openai_api_key =  os.environ["OPENAI_API_KEY"])
llm_mini = ChatOpenAI(model='gpt-3.5-turbo', openai_api_key =  os.environ["OPENAI_API_KEY"])

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [92]:
class Sentences(BaseModel):
    sentences: List[str]

parser = PydanticOutputParser(pydantic_object=Sentences)

prompt = PromptTemplate.from_template("""
Decompose the Passage below into clear and simple propositions, ensuring they are interpretable out of
context.
1. Split compound sentence into simple sentences. Maintain the original phrasing from the input
whenever possible.
2. For any named entity that is accompanied by additional descriptive information, separate this
information into its own distinct proposition.
3. Decontextualize the proposition by adding necessary modifier to nouns or entire sentences
and replacing pronouns (e.g., "it", "he", "she", "they", "this", "that") with the full name of the
entities they refer to.
4. Present the results as a list of strings, formatted in JSON.

Example:

Input: Title: ¯Eostre. Section: Theories and interpretations, Connection to Easter Hares. Content:
The earliest evidence for the Easter Hare (Osterhase) was recorded in south-west Germany in
1678 by the professor of medicine Georg Franck von Franckenau, but it remained unknown in
other parts of Germany until the 18th century. Scholar Richard Sermon writes that "hares were
frequently seen in gardens in spring, and thus may have served as a convenient explanation for the
origin of the colored eggs hidden there for children. Alternatively, there is a European tradition
that hares laid eggs, since a hare’s scratch or form and a lapwing’s nest look very similar, and
both occur on grassland and are first seen in the spring. In the nineteenth century the influence
of Easter cards, toys, and books was to make the Easter Hare/Rabbit popular throughout Europe.
German immigrants then exported the custom to Britain and America where it evolved into the
Easter Bunny."
Output: [ "The earliest evidence for the Easter Hare was recorded in south-west Germany in
1678 by Georg Franck von Franckenau.", "Georg Franck von Franckenau was a professor of
medicine.", "The evidence for the Easter Hare remained unknown in other parts of Germany until
the 18th century.", "Richard Sermon was a scholar.", "Richard Sermon writes a hypothesis about
the possible explanation for the connection between hares and the tradition during Easter", "Hares
were frequently seen in gardens in spring.", "Hares may have served as a convenient explanation
for the origin of the colored eggs hidden in gardens for children.", "There is a European tradition
that hares laid eggs.", "A hare’s scratch or form and a lapwing’s nest look very similar.", "Both
hares and lapwing’s nests occur on grassland and are first seen in the spring.", "In the nineteenth
century the influence of Easter cards, toys, and books was to make the Easter Hare/Rabbit popular
throughout Europe.", "German immigrants exported the custom of the Easter Hare/Rabbit to
Britain and America.", "The custom of the Easter Hare/Rabbit evolved into the Easter Bunny in
Britain and America."]

Return the result in the following JSON format:
{{"sentences": ["sentence1", "sentence2", ...]}}

Passage:
{text}
""")



In [93]:
# Make the LLM output structured
runnable = prompt | llm | parser
runnable_mini = prompt | llm_mini | parser

## Proposition on Paragraph

In [79]:
test_text_p = test_sections[2]
print(test_text_p)

<H2> Plot ( edit ) </H2> <P> Thomas , Newt , and Frypan are the last of the free `` Gladers '' immune to the Flare virus that has ravaged the world 's population . Against the orders of Vince , who now leads The Right Arm resistance , the three leave their camp to rescue their immune friend Minho , who is being tortured by the organization WCKD in hopes of developing a cure for the virus . Heading for the `` Last City '' , where WCKD 's headquarters is located , the group is attacked by Cranks , humans infected with the Flare . However , they are saved by Jorge and Brenda , who join them . </P> <P> The group makes it to the wall of the Last City , which protects the city from Cranks . Outside the wall , people are rallying in protest to be let into the city . When WCKD opens fire at the protesters , the group is captured by a contingent of masked men and are taken to a hideout . One of the masked men is revealed to be Gally , who has survived being killed by Minho at the end of The Maz

In [80]:
output = runnable.invoke(test_text_p)
for i,eachProp in enumerate( output.sentences):
  print(f"{i}: {eachProp}")

0: Thomas, Newt, and Frypan are the last of the free 'Gladers'.
1: The free 'Gladers' are immune to the Flare virus.
2: The Flare virus has ravaged the world's population.
3: Vince now leads The Right Arm resistance.
4: Vince orders Thomas, Newt, and Frypan not to leave their camp.
5: Thomas, Newt, and Frypan leave their camp against the orders of Vince.
6: Thomas, Newt, and Frypan leave their camp to rescue their friend Minho.
7: Minho is immune to the Flare virus.
8: The organization WCKD is torturing Minho.
9: WCKD hopes to develop a cure for the Flare virus by torturing Minho.
10: The 'Last City' is the location of WCKD's headquarters.
11: Thomas, Newt, and Frypan head for the 'Last City'.
12: The group is attacked by Cranks.
13: Cranks are humans infected with the Flare virus.
14: Jorge and Brenda save Thomas, Newt, and Frypan from the Cranks.
15: Jorge and Brenda join Thomas, Newt, and Frypan.
16: The group reaches the wall of the Last City.
17: The wall of the Last City protects

In [90]:
output = runnable_mini.invoke(test_text_p)
for i,eachProp in enumerate( output.sentences):
  print(f"{i}: {eachProp}")

0: Thomas, Newt, and Frypan are the last of the free Gladers immune to the Flare virus that has ravaged the world's population.
1: Vince leads The Right Arm resistance.
2: Vince issues orders against Thomas, Newt, and Frypan leaving their camp to rescue their immune friend Minho.
3: Minho is being tortured by the organization WCKD in hopes of developing a cure for the virus.
4: The group heads for the Last City where WCKD's headquarters is located.
5: The group is attacked by Cranks but saved by Jorge and Brenda.
6: The group makes it to the wall of the Last City which protects the city from Cranks.
7: Outside the wall, people are rallying in protest to be let into the city.
8: WCKD opens fire at the protesters.
9: The group is captured by a contingent of masked men and taken to a hideout.
10: One of the masked men is Gally who has survived being killed by Minho at the end of The Maze Runner.
11: Gally leads Thomas and Newt to Lawrence, a rebellion leader for those outside the walls.
1

## Proposition on List

In [81]:
# With List case:
test_text_list = test_sections[3]
print(test_text_list)

<H2> Cast ( edit ) </H2> <Ul> <Li> Dylan O'Brien as Thomas </Li> <Li> Kaya Scodelario as Teresa </Li> <Li> Thomas Brodie - Sangster as Newt </Li> <Li> Dexter Darden as Frypan </Li> <Li> Nathalie Emmanuel as Harriet </Li> <Li> Giancarlo Esposito as Jorge </Li> <Li> Aidan Gillen as Janson </Li> <Li> Walton Goggins as Lawrence </Li> <Li> Ki Hong Lee as Minho </Li> <Li> Jacob Lofland as Aris </Li> <Li> Katherine McNamara as Sonya </Li> <Li> Barry Pepper as Vince </Li> <Li> Will Poulter as Gally </Li> <Li> Rosa Salazar as Brenda </Li> <Li> Patricia Clarkson as Ava Paige </Li> </Ul>


In [82]:
output_list = runnable.invoke(test_text_list)
for i,eachProp in enumerate( output_list.sentences):
  print(f"{i}: {eachProp}")

0: Dylan O'Brien is cast in the role of Thomas.
1: Kaya Scodelario is cast in the role of Teresa.
2: Thomas Brodie-Sangster is cast in the role of Newt.
3: Dexter Darden is cast in the role of Frypan.
4: Nathalie Emmanuel is cast in the role of Harriet.
5: Giancarlo Esposito is cast in the role of Jorge.
6: Aidan Gillen is cast in the role of Janson.
7: Walton Goggins is cast in the role of Lawrence.
8: Ki Hong Lee is cast in the role of Minho.
9: Jacob Lofland is cast in the role of Aris.
10: Katherine McNamara is cast in the role of Sonya.
11: Barry Pepper is cast in the role of Vince.
12: Will Poulter is cast in the role of Gally.
13: Rosa Salazar is cast in the role of Brenda.
14: Patricia Clarkson is cast in the role of Ava Paige.


## Proposition on table

In [86]:
test_text_table = test_sections[0]
print(test_text_table)

Maze Runner : the Death Cure - wikipedia <H1> Maze Runner : the Death Cure </H1> Jump to : navigation , search For the book the film is based on , see The Death Cure . <Table> <Tr> <Th_colspan="2"> Maze Runner : The Death Cure </Th> </Tr> <Tr> <Td_colspan="2"> Theatrical release poster </Td> </Tr> <Tr> <Th> Directed by </Th> <Td> Wes Ball </Td> </Tr> <Tr> <Th> Produced by </Th> <Td> <Ul> <Li> Ellen Goldsmith - Vein </Li> <Li> Wyck Godfrey </Li> <Li> Marty Bowen </Li> <Li> Joe Hartwick , Jr . </Li> <Li> Wes Ball </Li> <Li> Lee Stollman </Li> </Ul> </Td> </Tr> <Tr> <Th> Screenplay by </Th> <Td> T.S. Nowlin </Td> </Tr> <Tr> <Th> Based on </Th> <Td> The Death Cure by James Dashner </Td> </Tr> <Tr> <Th> Starring </Th> <Td> <Ul> <Li> Dylan O'Brien </Li> <Li> Kaya Scodelario </Li> <Li> Thomas Brodie - Sangster </Li> <Li> Nathalie Emmanuel </Li> <Li> Giancarlo Esposito </Li> <Li> Aidan Gillen </Li> <Li> Walton Goggins </Li> <Li> Ki Hong Lee </Li> <Li> Barry Pepper </Li> <Li> Will Poulter </Li>

In [87]:
output_table = runnable.invoke(test_text_table)
for i,eachProp in enumerate( output_table.sentences):
  print(f"{i}: {eachProp}")

0: Maze Runner: The Death Cure is a title.
1: Wikipedia contains an article titled Maze Runner: The Death Cure.
2: The article Maze Runner: The Death Cure includes a section titled Theatrical release poster.
3: Wes Ball directed Maze Runner: The Death Cure.
4: The producers of Maze Runner: The Death Cure are Ellen Goldsmith-Vein, Wyck Godfrey, Marty Bowen, Joe Hartwick Jr., Wes Ball, and Lee Stollman.
5: Ellen Goldsmith-Vein is a producer.
6: Wyck Godfrey is a producer.
7: Marty Bowen is a producer.
8: Joe Hartwick Jr. is a producer.
9: Lee Stollman is a producer.
10: T.S. Nowlin wrote the screenplay for Maze Runner: The Death Cure.
11: Maze Runner: The Death Cure is based on the novel The Death Cure.
12: James Dashner wrote the novel The Death Cure.
13: The film Maze Runner: The Death Cure stars Dylan O'Brien, Kaya Scodelario, Thomas Brodie-Sangster, Nathalie Emmanuel, Giancarlo Esposito, Aidan Gillen, Walton Goggins, Ki Hong Lee, Barry Pepper, Will Poulter, and Patricia Clarkson.
14:

In [94]:
output_table = runnable_mini.invoke(test_text_table)
for i,eachProp in enumerate( output_table.sentences):
  print(f"{i}: {eachProp}")



0: Maze Runner : The Death Cure is a 2018 American dystopian science fiction action film directed by Wes Ball.
1: The film is based on the novel The Death Cure written by James Dashner.
2: The film is the sequel to the 2015 film Maze Runner : The Scorch Trials.
3: The film is the third and final installment in the Maze Runner film series.
4: The film stars Dylan O'Brien, Kaya Scodelario, Thomas Brodie - Sangster, Dexter Darden, Nathalie Emmanuel, Giancarlo Esposito, Aidan Gillen, Walton Goggins, Ki Hong Lee, Jacob Lofland, Katherine McNamara, Barry Pepper, Will Poulter, Rosa Salazar, and Patricia Clarkson.
5: The film was originally set to be released on February 17, 2017, in the United States by 20th Century Fox.
6: The studio rescheduled the film 's release for January 26, 2018 in theaters and IMAX.
7: The rescheduling allowed time for O'Brien to recover from injuries he sustained during filming.
8: The film received mixed reviews from critics.
9: The film has grossed over $272 milli


**Note**: Above tests cost on GPT-4 Turbo : around $0.15. GPT3Turbo is around the half